In [1]:
from fastai.tabular.all import *

from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb

In [2]:
import pickle

In [3]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [4]:
train_df = pd.read_feather('train_24cols.feather')

In [5]:
def train_models(train):
    # Hyperparammeters (optimized)
    seed = 29
    params = {
        'learning_rate': 0.1,        
        'lambda_l1': 2,
        'lambda_l2': 7,
        'num_leaves': 800,
        'min_sum_hessian_in_leaf': 20,
        'feature_fraction': 0.8,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.9,
        'bagging_freq': 42,
        'min_data_in_leaf': 700,
        'max_depth': 4,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
    }   
    
    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    models =[]
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create a KFold object
    #kfold = KFold(n_splits = 10, random_state = 1111, shuffle = True)
    kfold = GroupKFold(n_splits=5)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x, groups=train.time_id)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 3000, 
                          early_stopping_rounds = 25, 
                          verbose_eval = 100,
                          feval = feval_rmspe)
        models.append(model)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        #test_predictions += model.predict(x_test) / 10
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return models

In [6]:
models = train_models(train_df)

Training fold 1


/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1996: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1727: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1460: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000469637	training's RMSPE: 0.217544	valid_1's rmse: 0.000504737	valid_1's RMSPE: 0.232782
[200]	training's rmse: 0.000455118	training's RMSPE: 0.210818	valid_1's rmse: 0.000496213	valid_1's RMSPE: 0.228851
[300]	training's rmse: 0.000447024	training's RMSPE: 0.207069	valid_1's rmse: 0.000492359	valid_1's RMSPE: 0.227074
[400]	training's rmse: 0.000440887	training's RMSPE: 0.204226	valid_1's rmse: 0.000490766	valid_1's RMSPE: 0.226339
Early stopping, best iteration is:
[391]	training's rmse: 0.000441383	training's RMSPE: 0.204456	valid_1's rmse: 0.000490479	valid_1's RMSPE: 0.226207
Training fold 2
Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000469129	training's RMSPE: 0.216681	valid_1's rmse: 0.000506837	valid_1's RMSPE: 0.236454
[200]	training's rmse: 0.000454093	training's RMSPE: 0.209736	valid_1's rmse: 0.000500326	valid_1's RMSPE: 0.233417
Early stopping,

In [9]:
train_df

,log_return2_std_0_600,stock_id,row_id,time_id,target,log_return_price_std_0_600,order_count_sum_0_600,seconds_in_bucket_size_0_600,size_sum_0_600,log_return1_std_0_600_min_time,...,log_return1_std_200_600,log_return1_std_400_600,price_spread_mean_0_600,log_return_price_std_0_600_mean_time,log_return_price_std_200_600_mean_time,log_return_price_std_400_600_mean_time,log_return_price_std_0_600_min_time,log_return_price_std_200_600_min_time,log_return_price_std_400_600_min_time,total_volume_mean_0_600
0,0.000286,0,0-5,5,0.004136,0.000324,110.0,40.0,3179.0,0.000058,...,0.000170,0.000163,0.000865,0.000318,0.000315,0.000317,0.000074,0.000074,0.000075,312.733337
1,0.000101,0,0-11,11,0.001445,0.000168,57.0,30.0,1289.0,0.000021,...,0.000051,0.000066,0.000432,0.000214,0.000210,0.000195,0.000000,0.000000,0.000000,380.630005
2,0.000196,0,0-16,16,0.002168,0.000394,68.0,25.0,2161.0,0.000042,...,0.000097,0.000083,0.000733,0.000230,0.000229,0.000217,0.000062,0.000063,0.000066,417.441681
3,0.000149,0,0-31,31,0.002195,0.000399,59.0,15.0,1962.0,0.000014,...,0.000093,0.000071,0.000897,0.000267,0.000264,0.000246,0.000022,0.000022,0.000000,465.523346
4,0.000133,0,0-62,62,0.001747,0.000194,89.0,22.0,1791.0,0.000019,...,0.000079,0.000102,0.000401,0.000214,0.000204,0.000192,0.000031,0.000030,0.000032,332.621674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,0.000240,126,126-32751,32751,0.003461,0.000365,103.0,37.0,2570.0,0.000035,...,0.000169,0.000128,0.000870,0.000242,0.000246,0.000236,0.000048,0.000050,0.000055,418.473328
428928,0.000204,126,126-32753,32753,0.003113,0.000320,147.0,43.0,2323.0,0.000021,...,0.000188,0.000201,0.000670,0.000207,0.000211,0.000197,0.000027,0.000027,0.000028,222.258331
428929,0.000246,126,126-32758,32758,0.004070,0.000332,98.0,35.0,3740.0,0.000037,...,0.000141,0.000150,0.000754,0.000260,0.000261,0.000264,0.000000,0.000000,0.000000,345.883331
428930,0.000219,126,126-32763,32763,0.003357,0.000232,234.0,80.0,9389.0,0.000052,...,0.000149,0.000128,0.000526,0.000226,0.000225,0.000226,0.000060,0.000060,0.000061,437.748322


In [10]:
for idx, model in enumerate(models):
    filename = f'optiver-models-private/data/models/lgb24_fold{idx}.pickle'
    pickle.dump(model, open(filename, 'wb'))
    

In [6]:
x_test = train_df.drop(['row_id', 'time_id', 'target'], axis = 1)
x_test['stock_id'] = x_test['stock_id'].astype(int)

In [8]:
for idx in range(10):
    filename = f'models/lgb_fold{idx}.pickle'
    model = pickle.load(open(filename, 'rb'))
    preds = model.predict(x_test)
    print(rmspe(train_df.target, preds))

0.18470989403729027
0.1737223782065638
0.19073378463864546
0.1880587200715555
0.17799874022536552
0.1725530401092998
0.1898247989760112
0.18514039121932407
0.18203909027786244
0.1981542175208046


## NEW features

In [34]:
from optiver_features import *

In [38]:
book_feature_dict = {
        wap1: [np.mean, np.std],
        wap2: [np.mean, np.std],
        log_return1: [realized_volatility, np.mean, np.std],
        log_return2: [realized_volatility, np.mean, np.std],
        wap_balance: [np.mean, np.std],
        price_spread:[np.mean, np.std],
        bid_spread: [np.mean, np.std],
        ask_spread:[np.mean, np.std],
        total_volume:[np.mean, np.std],
        volume_imbalance:[np.mean, np.std]
    }

trade_feature_dict = {
        log_return_price: [realized_volatility],
        'seconds_in_bucket':[np.size],
        'size':[np.sum],
        'order_count':[np.mean],
    }

time_windows = [(0,600), (150,600), (300,600), (450,600)]


cols = ['log_return_price_realized_volatility', 'log_return1_realized_volatility', 'log_return2_realized_volatility']
time_id_features = [f'{col}_{x}_{y}' for x,y in time_windows for col in cols] 
time_id_aggregations = ['mean', 'std', 'max', 'min' ]
stock_id_features = time_id_features
stock_id_aggregations = time_id_aggregations

In [40]:
time_id_features

['log_return_price_realized_volatility_0_600',
 'log_return1_realized_volatility_0_600',
 'log_return2_realized_volatility_0_600',
 'log_return_price_realized_volatility_150_600',
 'log_return1_realized_volatility_150_600',
 'log_return2_realized_volatility_150_600',
 'log_return_price_realized_volatility_300_600',
 'log_return1_realized_volatility_300_600',
 'log_return2_realized_volatility_300_600',
 'log_return_price_realized_volatility_450_600',
 'log_return1_realized_volatility_450_600',
 'log_return2_realized_volatility_450_600']

In [41]:
ofg = OptiverFeatureGenerator(book_feature_dict, trade_feature_dict, time_windows, time_id_features,time_id_aggregations, stock_id_features, stock_id_aggregations)

In [42]:
%%time
train_new = ofg.generate_train_df()

CPU times: user 2.68 s, sys: 1.12 s, total: 3.8 s
Wall time: 3min 33s


In [43]:
models = train_models(train_new)

Training fold 1


/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000470255	training's RMSPE: 0.21783	valid_1's rmse: 0.000506402	valid_1's RMSPE: 0.23355
[200]	training's rmse: 0.000452357	training's RMSPE: 0.209539	valid_1's rmse: 0.000501532	valid_1's RMSPE: 0.231304
Early stopping, best iteration is:
[232]	training's rmse: 0.000448597	training's RMSPE: 0.207798	valid_1's rmse: 0.000501053	valid_1's RMSPE: 0.231083
Training fold 2


/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000469125	training's RMSPE: 0.216679	valid_1's rmse: 0.000517973	valid_1's RMSPE: 0.24165
[200]	training's rmse: 0.000451591	training's RMSPE: 0.20858	valid_1's rmse: 0.000512826	valid_1's RMSPE: 0.239249
[300]	training's rmse: 0.000439905	training's RMSPE: 0.203183	valid_1's rmse: 0.000510238	valid_1's RMSPE: 0.238041
Early stopping, best iteration is:
[322]	training's rmse: 0.000438094	training's RMSPE: 0.202346	valid_1's rmse: 0.000509593	valid_1's RMSPE: 0.23774
Training fold 3


...

Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000472924	training's RMSPE: 0.21857	valid_1's rmse: 0.000483791	valid_1's RMSPE: 0.22515
[200]	training's rmse: 0.000455384	training's RMSPE: 0.210463	valid_1's rmse: 0.000481298	valid_1's RMSPE: 0.22399
Early stopping, best iteration is:
[263]	training's rmse: 0.000447562	training's RMSPE: 0.206849	valid_1's rmse: 0.000480469	valid_1's RMSPE: 0.223604
Training fold 4


/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000473982	training's RMSPE: 0.219322	valid_1's rmse: 0.000483915	valid_1's RMSPE: 0.224138
Early stopping, best iteration is:
[126]	training's rmse: 0.000468624	training's RMSPE: 0.216843	valid_1's rmse: 0.00048298	valid_1's RMSPE: 0.223705
Training fold 5


/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/slex/programy/anaconda3/envs/fastai/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	training's rmse: 0.000472782	training's RMSPE: 0.219407	valid_1's rmse: 0.00048239	valid_1's RMSPE: 0.2208
Early stopping, best iteration is:
[151]	training's rmse: 0.000462582	training's RMSPE: 0.214673	valid_1's rmse: 0.000481369	valid_1's RMSPE: 0.220333
Our out of folds RMSPE is 0.22738050185706846


In [44]:
for idx, model in enumerate(models):
    filename = f'optiver-models-private/data/models/lgb_fold{idx}.pickle'
    pickle.dump(model, open(filename, 'wb'))

In [46]:
train_new['seconds_in_bucket_size_0_600']=train_new['seconds_in_bucket_size_0_600'].astype(np.float32)

In [51]:
nacols = train_new.isna().any()

In [53]:
nacols[nacols==True]

seconds_in_bucket_size_0_600                    True
size_sum_0_600                                  True
order_count_mean_0_600                          True
log_return_price_realized_volatility_0_600      True
seconds_in_bucket_size_150_600                  True
size_sum_150_600                                True
order_count_mean_150_600                        True
log_return_price_realized_volatility_150_600    True
seconds_in_bucket_size_300_600                  True
size_sum_300_600                                True
order_count_mean_300_600                        True
log_return_price_realized_volatility_300_600    True
seconds_in_bucket_size_450_600                  True
size_sum_450_600                                True
order_count_mean_450_600                        True
log_return_price_realized_volatility_450_600    True
dtype: bool

In [48]:
[(c, train_new[c].dtype) for c in train_new.columns]

[('stock_id', dtype('int64')),
 ('time_id', dtype('int64')),
 ('target', dtype('float64')),
 ('row_id', dtype('O')),
 ('seconds_in_bucket_size_0_600', dtype('float32')),
 ('size_sum_0_600', dtype('float64')),
 ('order_count_mean_0_600', dtype('float32')),
 ('log_return_price_realized_volatility_0_600', dtype('float32')),
 ('seconds_in_bucket_size_150_600', dtype('float32')),
 ('size_sum_150_600', dtype('float64')),
 ('order_count_mean_150_600', dtype('float32')),
 ('log_return_price_realized_volatility_150_600', dtype('float32')),
 ('seconds_in_bucket_size_300_600', dtype('float32')),
 ('size_sum_300_600', dtype('float64')),
 ('order_count_mean_300_600', dtype('float32')),
 ('log_return_price_realized_volatility_300_600', dtype('float32')),
 ('seconds_in_bucket_size_450_600', dtype('float32')),
 ('size_sum_450_600', dtype('float64')),
 ('order_count_mean_450_600', dtype('float32')),
 ('log_return_price_realized_volatility_450_600', dtype('float32')),
 ('wap1_mean_0_600', dtype('float32